In [1]:
from scrawl import *
import random
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse


c = Crawl() # Creating object

# Creating list of proxies
proxy_col = dashboard['proxies']
proxy = proxy_col.find({'country':'China'})
proxy_list = list(map(lambda x:x['ip'],list(proxy)))

def get_proxy():
    
    return {'http': random.choice(proxy_list)}


def download_image(image_url,image_path):

    time.sleep(random.randint(1,3))
    try:
        r = requests.get(image_url,proxies=get_proxy(),timeout=120)
        response_code = r.status_code
        if response_code != 200:
            return f'Unable to fetch image {image_url} Error code: {r.status_code}\n'
            # print(f'Unable to fetch image {image_url} Error code: {r.status_code}')
        else:
            r = r.content
            with open(image_path, 'wb') as f:
                f.write(r)
    except:
        return f'Unable to fetch image {image_url} Unknown error\n'
        print(f'Unable to fetch image {image_url} Unknown error\n')

    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5eb16c2db378671cf746ebb8'  
site = 's_weibo_com_user'
cl_data = dashboard['core_weibo']

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_username = 0
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_download_image = 0
publish_source = 's.weibo.com'
country = 'China'
language = 'Chinese'
cnt = 0
publish_time = '00:00:00'
pub_date = created_on  # no pattern to find date
# Hence we will concate date(which is in a sentence) in the source content.
no_of_not_working_queries = 0
headers = {
    'authority': 'weibo.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '^\\^',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
    'cookie': 'SINAGLOBAL=764815322341.5566.1622097283265; SUB=2AkMXj8zTf8NxqwJRmP0RzmrjaY1yyg3EieKh0z0IJRMxHRl-yT92qmgntRB6PA_iPI199P4zlRz9zonVc5W23plzUH7V; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9W55o9Nf.NuDNjNQuIS8pJY; _s_tentry=-; Apache=3847225399074.1636.1624690011593; ULV=1624690011604:5:4:4:3847225399074.1636.1624690011593:1624608998989',
}

user_links = ['https://weibo.com/globaltimescn?refer_flag=1001030103_&ssl_rnd=1623643790.2876',
             'https://weibo.com/newsxh?refer_flag=1001030103_&is_all=1&ssl_rnd=1625039615.9147',
             'https://weibo.com/u/7547570847?ssl_rnd=1625044773.0251&is_all=1',
             'https://weibo.com/hebgqt?refer_flag=1001030103_&is_all=1&is_all=1&ssl_rnd=1625039615.9147',
             'https://weibo.com/xhstyb?refer_flag=1001030103_',
             'https://weibo.com/cctvnewsbeijing?ssl_rnd=1625044839.2109&is_all=1',
             'https://weibo.com/u/3704995720?refer_flag=1001030103_',
             'https://weibo.com/badnewscn?nick=%E4%B8%8A%E6%B5%B7%E4%B9%90%E6%A2%A6_Bad_News_China',
             'https://weibo.com/chinadailywebsite?ssl_rnd=1623648346.4715',
             'https://weibo.com/chinadailywebsite?ssl_rnd=1623648346.4715&is_all=1',
             'https://weibo.com/weather01?refer_flag=1001030103_',
             'https://weibo.com/cctvnewsbeijing?refer_flag=1001030103_',
             'https://weibo.com/cctvnewsbeijing?refer_flag=1001030103_&ssl_rnd=1623648683.3943&is_all=1',
             'https://weibo.com/baincompany?refer_flag=1001030103_&ssl_rnd=1623649023.9775&is_all=1',
             'https://weibo.com/u/6139776919?refer_flag=1001030103_',
             'https://weibo.com/ttarticle/p/show?id=2309404621567283036299',
             'https://weibo.com/zhongguowang?refer_flag=1001030103_',
             'https://weibo.com/chinadailywebsite?ssl_rnd=1623648346.4715&is_all=1',
             'https://weibo.com/cctv5?ssl_rnd=1625045238.3648&is_all=1',
             'https://weibo.com/cgtnenespanol?refer_flag=1001030103_',
             'https://weibo.com/chinanewsv?ssl_rnd=1625045298.6861',
             'https://weibo.com/cctvnewsbeijing?refer_flag=1001030103_&is_all=1',
             'https://weibo.com/cctvnewsbeijing?refer_flag=1001030103_&ssl_rnd=1623650973.4547&is_all=1',
             'https://weibo.com/cctvnewsbeijing?refer_flag=1001030103_&ssl_rnd=1623650973.4547&is_all=1',
             'https://weibo.com/gxrb2013?refer_flag=1001030103_&ssl_rnd=1623654240.3591',
             'https://weibo.com/gmrb1949?ssl_rnd=1623654193.1401s',
             'https://weibo.com/beijingdaily?ssl_rnd=1625045336.662',
             'https://weibo.com/thepapernewsapp?nick=%E6%BE%8E%E6%B9%83%E6%96%B0%E9%97%BB',
             'https://s.weibo.com/claim/apply?object_id=1022:231522de583bd8b6ab3e27312f5d2d87d081ea&uid=0&query=%23%E7%8E%AF%E7%90%83%E6%97%B6%E6%8A%A5%23',
             'https://weibo.com/u/6314326330?ssl_rnd=1625045361.6221',
             'https://weibo.com/shinebridal?is_all=1']

foot_fall = c.get_foot_fall(publish_source)

for user_link in user_links:
    if '&ssl_rnd' not in user_link:
        user_link += '&ssl_rnd=1625039615.9147'
    logger.info(f'Fetching cat url  {user_link}\n') 
    try:
#         print(user_link)
        user_page = requests.get(user_link,headers = headers ,timeout=200).text
        time.sleep(3)
    except:
        print("time error")
    if user_page.startswith('Unable to fetch'):
        logger.info(user_page) # writes error message with error code
        no_of_not_working_queries += 1
        continue   
    
    for i in user_page.split('WB_feed_detail clearfix')[1:]:
        # Date and time 
        date_time = c.scrap('<a\s*name=.*?title=.*?"(.*?)"', i) 
        date_time  = re.sub(r'\\','',date_time)  # r is used to remove this special symbol (\) from the string
        date_time = c.strip_html(date_time)
#         print(date_time)
        source_content = c.scrap('<div\s*class=.*?"WB_text W_f14.*?".*?>(.*?)<div.*?class=.*?"WB_expand_media_box', i)
        if not source_content:
            source_content = c.scrap('<div\s*class=.*?"WB_text W_f14.*?".*?>(.*?)<a\s*suda-uatrack=', i) + '\n' + date_time
        source_content = c.strip_html(source_content)
        source_content = re.sub(r'\\n','',source_content,re.S)  # this is how to subsitute the special symbol using regex,  use prefix (r) with it
        source_content = re.sub(r'\\','',source_content,re.S)
       

        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {user_link}\n')
            skipped_due_to_content += 1
            continue
        # handle duplicates
        content_query = {'content':source_content}
        dic = cl_data.find_one(content_query,{'content': 1}) 
        if dic:
            duplicate_data += 1
            continue

#         # source_headline
        source_headline = source_content
    
#        user name
        username = c.scrap('nick=(.*?)\&', i)

        # skip if username not found
        if not username:
            logger.info(f'Skipping due to username {user_link}\n')
            skipped_due_to_username += 1
            continue  
#       # journalist
        journalist = ''
        if not journalist: journalist = 'NA'

        # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
        images_path = []
        
        favourites = c.scrap('click:fav">.*?>(.*?)<', i)
        favourites = c.scrap('(\d+)', favourites)
        
        forward = c.scrap('click:repost.*?>(.*?)<', i)
        forward = c.scrap('(\d+)', forward)
        
        comments = c.scrap('click:comment.*?>(.*?)<', i)
        comments = c.scrap('(\d+)', comments)        
        
        likes = c.scrap('click:like.*?<em>(.*?)</em>', i)
        likes = c.scrap('(\d+)', likes)
        #  this is for 
#         img_urls = re.findall('<li.*?<img\s*src=.*?"(.*?)"', i, flags=re.S)
# #         img_urls = re.findall('<li.*?>\s*<img\s*src="(.*?)"', i, flags=re.S)
#         for img_url in img_urls:
        
#             if 'admin' in img_url:
#                 continue
        # Image
        img_urls = re.findall('<li\s*class=.*?"WB_pic\s*li_1\s*S_bg1\s*S_line2\s*bigcursor.*?<img\s*src=.*?"(.*?)"', i, flags=re.S)
#         img_urls = re.findall('<li.*?>\s*<img\s*src="(.*?)"', i, flags=re.S)
        for img_url in img_urls:
            img_url  = re.sub(r'\\','',img_url)
            img_name = c.scrap('.*/(.*)', img_url)
            img_url  = re.sub(r'\\','',img_url)
            img_path = f'{image_directory}/{img_name}'
            if not img_url.startswith('http'):
                img_url = 'https:' + img_url

            # if image is not downloaded return an error message
            download_message = download_image(img_url, img_path)  
            images_path.append(img_path)
            
            if download_message:
                logger.info(download_message)  # writes error message with error code
                unable_to_download_image += 1
                continue  
     
         # storing the above data in a dictionary
        clientdata = {
                        "client_master": client_id,
                        "articleid": client_id,
                        "medium": "Weibo",
#                         "searchkeyword": [query], 
                        "entityname": [], 
                        "process_flage": "1",
                        "na_flage": "0",
                        "na_reason": "",
                        "qc_by": "",
                        "qc_on": "",
                        "location": "",
                        "spokeperson": "",
                        "quota": "",
                        "overall_topics": "",
                        "person": "",
                        "overall_entites": "",
                        "overall_tonality": "",
                        "overall_wordcount": word_count,
                        "article_subjectivity": "",
                        "article_summary": "",
                        "pub_date": pub_date,
                        "publish_time": publish_time,
                        "harvest_time": harvest_time,  
                        "temp_link": user_link,
                        "publish_source": username,  
                        "programme": "null",
                        "feed_class": "",
                        "publishing_platform": "Weibo",
                        "klout_score": "0",
                        "journalist": "", 
                        "headline": source_content,  
                        "content": source_content,  
                        "language": language,
                        "location_mention": "",
                        "source_link":user_link,
                        "author_contact": "",
                        "author_emailid": "",
                        "author_url": "",
                        "city": "",
                        "state": "",
                        "country": country,
                        "source": username,
                        "foot_fall": "",
                        "created_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        "active": "1",
                        "circulation": "0",
                        'favourites':favourites,
                        'forward': forward,
                        'comments':comments,
                        'likes': likes,
                        'images_path': images_path
        }

        cl_data.insert_one(clientdata)  
        no_of_data += 1
        

logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
# logger.info(f'Total number of queries: {len(queries)}\n')
logger.info(f'No. of not working queries: {no_of_not_working_queries}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Skipped due to username: {skipped_due_to_username}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Unable to download image: {unable_to_download_image}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

time error
time error


In [21]:
# checking the demo


def download_image(image_url,image_path):

    time.sleep(random.randint(1,3))
    r = requests.get(image_url,timeout=120)
    response_code = r.status_code
    if response_code != 200:
        return f'Unable to fetch image {image_url} Error code: {r.status_code}\n'
        # print(f'Unable to fetch image {image_url} Error code: {r.status_code}')
    else:
        r = r.content
        with open(image_path, 'wb') as f:
            f.write(r)
for img_url in img_urls:
    img_url  = re.sub(r'\\','',img_url)
    print(img_url)

    img_url  = re.sub(r'\\','',img_url)
    img_name = c.scrap('.*/(.*)', img_url)
    print(img_name)
    img_path = f'{image_directory}/{img_name}'
    if not img_url.startswith('http'):
        img_url = 'https:' + img_url
    print(img_url)
#     continue
    # if image is not downloaded return an error message
    download_message = download_image(img_url, img_path)  
    images_path.append(img_path)

    if download_message:
        logger.info(download_message)  # writes error message with error code
        unable_to_download_image += 1
        continue  

//wx1.sinaimg.cn/orj360/008eMPXFly1gnzqmwfzwgj30u01szqdz.jpg
008eMPXFly1gnzqmwfzwgj30u01szqdz.jpg
https://wx1.sinaimg.cn/orj360/008eMPXFly1gnzqmwfzwgj30u01szqdz.jpg
//wx3.sinaimg.cn/orj360/008eMPXFly1gs2q8ihncpj31670u0qfo.jpg
008eMPXFly1gs2q8ihncpj31670u0qfo.jpg
https://wx3.sinaimg.cn/orj360/008eMPXFly1gs2q8ihncpj31670u0qfo.jpg
//wx1.sinaimg.cn/orj360/008eMPXFly1gs66kac3b8j30j60eejsf.jpg
008eMPXFly1gs66kac3b8j30j60eejsf.jpg
https://wx1.sinaimg.cn/orj360/008eMPXFly1gs66kac3b8j30j60eejsf.jpg
//wx2.sinaimg.cn/orj360/008eMPXFly1gs66hteg7uj30hy0tn43d.jpg
008eMPXFly1gs66hteg7uj30hy0tn43d.jpg
https://wx2.sinaimg.cn/orj360/008eMPXFly1gs66hteg7uj30hy0tn43d.jpg
//wx3.sinaimg.cn/orj360/008eMPXFly1gs5u2gs85ij30j60csmxo.jpg
008eMPXFly1gs5u2gs85ij30j60csmxo.jpg
https://wx3.sinaimg.cn/orj360/008eMPXFly1gs5u2gs85ij30j60csmxo.jpg
//wx1.sinaimg.cn/orj360/008eMPXFly1gs2q21qi09j31660u0drd.jpg
008eMPXFly1gs2q21qi09j31660u0drd.jpg
https://wx1.sinaimg.cn/orj360/008eMPXFly1gs2q21qi09j31660u0drd.jpg
//wx3.sina

<!doctype html>
<html >
<head>
<meta charset="utf-8">
<meta content="CGTN，CGTN的微博，微博，新浪微博，weibo" name="keywords" />
<meta content="CGTN，中国国际电视台官方微博。CGTN的微博主页、个人资料、相册。新浪微博，随时随地分享身边的新鲜事儿。" name="description" />
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta name="renderer" content="webkit" />
<meta name="viewport" content="initial-scale=1,minimum-scale=1" />
<link rel="dns-prefetch" href="//img.t.sinajs.cn/">
<link rel="dns-prefetch" href="//img1.t.sinajs.cn/">
<link rel="dns-prefetch" href="//js.t.sinajs.cn/">
<link rel="dns-prefetch" href="//js1.t.sinajs.cn/">
<link rel="dns-prefetch" href="//js2.t.sinajs.cn/">
<link rel="dns-prefetch" href="//biz.weibo.com/">
<link rel="dns-prefetch" href="//beacon.sina.com.cn/">
<link rel="dns-prefetch" href="//rs.sinajs.cn/">
<link rel="dns-prefetch" href="//tp1.sinaimg.cn/">
<link rel="dns-prefetch" href="//tp2.sinaimg.cn/">
<link rel="dns-prefetch" href="//tp3.sinaimg.cn/">
<link rel="dns-prefetch" href="//tp4.sinaimg.cn/">
<link re

In [ ]:
# headers = {
#     'authority': 'weibo.com',
#     'cache-control': 'max-age=0',
#     'sec-ch-ua': '^\\^',
#     'sec-ch-ua-mobile': '?0',
#     'upgrade-insecure-requests': '1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
#     'sec-fetch-site': 'same-origin',
# #     'sec-fetch-mode': 'navigate',
#     'sec-fetch-user': '?1',
#     'sec-fetch-dest': 'document',
#     'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
#     'cookie': 'SINAGLOBAL=764815322341.5566.1622097283265; SUB=2AkMXj8zTf8NxqwJRmP0RzmrjaY1yyg3EieKh0z0IJRMxHRl-yT92qmgntRB6PA_iPI199P4zlRz9zonVc5W23plzUH7V; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9W55o9Nf.NuDNjNQuIS8pJY; _s_tentry=-; Apache=3847225399074.1636.1624690011593; ULV=1624690011604:5:4:4:3847225399074.1636.1624690011593:1624608998989',
# }

In [7]:
# headers = {
#     'authority': 'weibo.com',
#     'cache-control': 'max-age=0',
#     'sec-ch-ua': '^\\^',
#     'sec-ch-ua-mobile': '?0',
#     'upgrade-insecure-requests': '1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
#     'sec-fetch-site': 'same-origin',
#     'sec-fetch-mode': 'navigate',
#     'sec-fetch-user': '?1',
#     'sec-fetch-dest': 'document',
#     'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
#     'cookie': 'SINAGLOBAL=764815322341.5566.1622097283265; SUB=2AkMXj8zTf8NxqwJRmP0RzmrjaY1yyg3EieKh0z0IJRMxHRl-yT92qmgntRB6PA_iPI199P4zlRz9zonVc5W23plzUH7V; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9W55o9Nf.NuDNjNQuIS8pJY; _s_tentry=-; Apache=3847225399074.1636.1624690011593; ULV=1624690011604:5:4:4:3847225399074.1636.1624690011593:1624608998989',
# }

In [3]:
# import requests
# # old
# headers = {
#     'authority': 'weibo.com',
#     'cache-control': 'max-age=0',
#     'sec-ch-ua': '^\\^',
#     'sec-ch-ua-mobile': '?0',
#     'upgrade-insecure-requests': '1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
#     'sec-fetch-site': 'same-origin',
#     'sec-fetch-mode': 'navigate',
#     'sec-fetch-user': '?1',
#     'sec-fetch-dest': 'document',
#     'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
#     'cookie': 'SINAGLOBAL=764815322341.5566.1622097283265; SUB=2AkMXj8zTf8NxqwJRmP0RzmrjaY1yyg3EieKh0z0IJRMxHRl-yT92qmgntRB6PA_iPI199P4zlRz9zonVc5W23plzUH7V; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9W55o9Nf.NuDNjNQuIS8pJY; _s_tentry=-; Apache=3847225399074.1636.1624690011593; ULV=1624690011604:5:4:4:3847225399074.1636.1624690011593:1624608998989',
# }
# # new
# # headers = {
# #     'authority': 'weibo.com',
# #     'cache-control': 'max-age=0',
# #     'sec-ch-ua': '^\\^',
# #     'sec-ch-ua-mobile': '?0',
# #     'upgrade-insecure-requests': '1',
# #     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
# #     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
# #     'sec-fetch-site': 'same-origin',
# #     'sec-fetch-mode': 'navigate',
# #     'sec-fetch-user': '?1',
# #     'sec-fetch-dest': 'document',
# #     'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
# #     'cookie': 'SUB=2AkMXhfNwf8NxqwJRmfwVzGvgaY9yzA7EieKh2QKrJRMxHRl-yT92qlIQtRB6PAXdnzM570CXEl986-xQCVun_eaNAKXg; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9W5lqI2D669KTv.cS2P1pE_; SINAGLOBAL=9491775771871.78.1624865871380; _s_tentry=-; Apache=937755205172.3126.1625039308492; ULV=1625039308522:2:2:2:937755205172.3126.1625039308492:1624865871399; httpsupgrade_ab=SSL',
# # }

# params = (
#     ('ssl_rnd', '1625039615.9147')
# )

# # response = requests.get('https://weibo.com/thepapernewsapp', headers=headers, params=params)

# #NB. Original query string below. It seems impossible to parse and
# #reproduce query strings 100% accurately so the one below is given
# #in case the reproduced version is not "correct".
# #https://weibo.com/thepapernewsapp?nick=%E6%BE%8E%E6%B9%83%E6%96%B0%E9%97%BB
# response = requests.get('https://weibo.com/thepapernewsapp?nick=^%^E6^%^BE^%^8E^%^E6^%^B9^%^83^%^E6^%^96^%^B0^%^E9^%^97^%^BB&ssl_rnd=1625039615.9147', headers=headers)
# # response = requests.get('https://weibo.com/thepapernewsapp?nick=%E6%BE%8E%E6%B9%83%E6%96%B0%E9%97%BB', headers=headers)

NameError: name 'response' is not defined